In [2]:
import numpy as np


In [3]:
source_map = np.array(list("1110001110001110011001100")).astype(np.int32)

source_map = source_map.reshape(5, 5)

print("original data")
print(source_map)

original data
[[1 1 1 0 0]
 [0 1 1 1 0]
 [0 0 1 1 1]
 [0 0 1 1 0]
 [0 1 1 0 0]]


In [6]:
# filter

filter1 = np.array(list('101010101')).astype(np.int32).reshape(3, 3)
print("filter")
print(filter1)



filter
[[1 0 1]
 [0 1 0]
 [1 0 1]]


# Convolution without bias

In [9]:
width_idx = height_idx = source_map.shape[0] - filter1.shape[0] + 1
result = np.zeros((width_idx, height_idx))

for i in range(width_idx):
    conv_width = filter1.shape[0] + i
    for j in range(height_idx):
        conv_height = filter1.shape[1] + j
        value1 = source_map[i:conv_width, j:conv_height] * filter1
        result[i, j] = np.sum(value1)

print(result)


[[4. 3. 4.]
 [2. 4. 3.]
 [2. 3. 4.]]


# Convolution with bias 

In [21]:
origin_width = origin_width = source_map.shape[0]
width_bias = height_bias = origin_width + 2 * 1 # right + left and up + down
source_map_bias = np.zeros((width_bias, height_bias))
for i in range(1, width_bias-1):
    for j in range(1, height_bias-1):
        source_map_bias[i, j] = source_map[i- 1, j - 1] 

print(source_map_bias)

width_idx = height_idx = source_map_bias.shape[0] - filter1.shape[0] + 1
result = np.zeros((width_idx, height_idx))


for i in range(width_idx):
    conv_width = filter1.shape[0] + i
    for j in range(height_idx):
        conv_height = filter1.shape[1] + j
        value1 = source_map_bias[i:conv_width, j:conv_height] * filter1
        result[i, j] = np.sum(value1)
result

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


array([[2., 2., 3., 1., 1.],
       [1., 4., 3., 4., 1.],
       [1., 2., 4., 3., 3.],
       [1., 2., 3., 4., 1.],
       [0., 2., 2., 1., 1.]])

Use scipy to calculate the convolution
====
____

In [22]:
from scipy.signal import convolve2d

convolve2d(source_map_bias, filter1 , mode="valid")

array([[2., 2., 3., 1., 1.],
       [1., 4., 3., 4., 1.],
       [1., 2., 4., 3., 3.],
       [1., 2., 3., 4., 1.],
       [0., 2., 2., 1., 1.]])

## Define the image transfomration for convolution
____

In [11]:
from skimage.exposure import rescale_intensity


def convolve(image:np.ndarray, kernel:np.ndarray):
    (iH, iW) = image.shape[:2]
    (kH, kW) = kernel.shape[:2]
    
    pad = int((kW - 1) / 2)
    image = cv2.copyMakeBorder(image, pad, pad, pad, pad, cv2.BORDER_REPLICATE)
    output = np.zeros((iH, iW), dtype="float32")
    
    for y in np.arange(pad, iH + pad):
        for x in np.arange(pad, iW + pad):
            roi = image[y-pad: y+pad+1, x-pad:x+pad+1]
            k = (roi * kernel).sum()
            output[y-pad, x-pad] = k
    output = rescale_intensity(output, in_range=(0, 255))
    output = (output * 255).astype("uint8")

    return output
    

In [7]:
import skimage
import cv2 

image = skimage.data.chelsea()

cv2.imshow("origin", image)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("gray", gray)

cv2.waitKey(0)
cv2.destroyAllWindows()


Blur 
===
----

In [14]:
smallBlur = np.ones((7, 7), dtype="float") * (1.0 / (7 * 7))

# convolution 
convolveOutput = convolve(gray, smallBlur)
cv2.imshow("little Blur", convolveOutput)
# opencvOputput = cv2.filter2D(gray, -1, smallBlur)
# cv2.imshow("cv2", opencvOputput)
largeBlur = np.ones((21, 21), dtype="float") * (1.0 / (21 * 21))

convolveOutput = convolve(gray, smallBlur)
cv2.imshow("Large Blur", convolveOutput)



cv2.waitKey(0)
cv2.destroyAllWindows()

Sharpen 
===
---

In [15]:
sharpen = np.array([
    [ 0, -1,  0],
    [-1,  5, -1],
    [ 0, -1,  0], 
], dtype="int")

convolveOutput = convolve(gray, sharpen)

cv2.imshow("sharpen", convolveOutput)

cv2.waitKey(0)
cv2.destroyAllWindows()

Laplacian 
===
----

In [16]:
laplacian = np.array(
   [[ 0,  1,  0],
    [ 1, -4,  1],
    [ 0,  1,  0]], dtype="int"
)

convolveOutput = convolve(gray, laplacian)
cv2.imshow("Laplacian", convolveOutput)

cv2.waitKey(0)
cv2.destroyAllWindows()

Sobel X-axis
===
----

In [20]:
sobel_x = np.array(
   [[ 1,  2,  1],
    [ 0,  0,  0],
    [-1, -2, -1]], dtype="int"
)

convolveOutput = convolve(gray, sobel_x)
cv2.imshow("Laplacian", convolveOutput)

cv2.waitKey(0)
cv2.destroyAllWindows()

Sobel Y-axis
====
-----

In [19]:
sobel_y = np.array(
   [[ 1,  0, -1],
    [ 2,  0, -2],
    [ 1,  0, -1]], dtype="int"
)


convolveOutput = convolve(gray, sobel_y)
cv2.imshow("Laplacian", convolveOutput)

cv2.waitKey(0)
cv2.destroyAllWindows()